In [1]:
try:
  import torch
  import torcchvision
  assert int(torch.__version__.split(".")) >=12, "torch veraion should be 1.12+"
  assert int(torchvision.__version__.split(".")) >=0.13, "torchvision version should be 0.13+"
  print(f"torch version:{torch.__version__}")
  print(f"torchvision version:{torchvision.__version__}")
except:
  print(f"[INFO] torch/torchvision version not correct")
  !pip3 install -U torch torchvision
  import torch
  import torchvision
  print(f"torch version:{torch.__version__}")
  print(f"torchvision version:{torchvision.__version__}")

[INFO] torch/torchvision version not correct
torch version:2.5.1+cu121
torchvision version:0.20.1+cu121


In [2]:
import torch
import matplotlib
import torchvision
from torch import nn
from torchvision import transforms

try:
  from torchinfo import summary
except:
  print("[INFO] torchinfo not found, installing it...")
  !pip install -q torchinfo
  from torchinfo import summary


try:
  from going_modular import data_setup, engine
except:
  print("[INFO] going_modular not found, installing it...")
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine

[INFO] torchinfo not found, installing it...
[INFO] going_modular not found, installing it...
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 4356 (delta 213), reused 253 (delta 177), pack-reused 4035 (from 1)
Receiving objects: 100% (4356/4356), 654.51 MiB | 36.64 MiB/s, done.
Resolving deltas: 100% (2584/2584), done.
Updating files: 100% (248/248), done.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!git clone https://github.com/mrdbourke/pytorch-deep-learning

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 4356 (delta 213), reused 253 (delta 177), pack-reused 4035 (from 1)
Receiving objects: 100% (4356/4356), 654.51 MiB | 40.95 MiB/s, done.
Resolving deltas: 100% (2584/2584), done.
Updating files: 100% (248/248), done.


In [6]:
!mv pytorch-deep-learning/going_modular .

mv: cannot move 'pytorch-deep-learning/going_modular' to './going_modular': Directory not empty


In [7]:
!rm -rf pytorch-deep-learning

In [8]:
import nibabel as nib
import numpy as np

subjetos = ["sub-RC4101", "sub-RC4103", "sub-RC4106", "sub-RC4107"]
sesiones = ["ses-1", "ses-2"]
imag_norm = []
for sujeto in subjetos:
    for sesion in sesiones:
        ruta = f"drive/MyDrive/Colab Notebooks/output/derivatives/preprocessed/{sujeto}/{sesion}/anat/{sujeto}_{sesion}_T1w_brain_registered.nii.gz"
        img1= nib.load(ruta)
        imag_norm.append(img1.shape)

imag_norm


[(91, 109, 91),
 (91, 109, 91),
 (91, 109, 91),
 (91, 109, 91),
 (91, 109, 91),
 (91, 109, 91),
 (91, 109, 91),
 (91, 109, 91)]

In [9]:
import random

base="drive/MyDrive/Colab Notebooks/output/derivatives/preprocessed"
imag_dir = []
for sujeto in subjetos:
  for sesion in sesiones:
    ruta = f"{base}/{sujeto}/{sesion}/anat/{sujeto}_{sesion}_T1w_brain_registered.nii.gz"
imag_dir.append(ruta)

random.shuffle(imag_dir)
train_ratio = 0.75
train_size = int(len(imag_dir) * train_ratio)
train= imag_dir[:train_size]
test = imag_dir[train_size:]



In [14]:
import os
import random
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

subjetos = ["sub-RC4101", "sub-RC4103", "sub-RC4106", "sub-RC4107"]
sesiones = ["ses-1", "ses-2"]
base_path = "drive/MyDrive/Colab Notebooks/output/derivatives/preprocessed"


image_paths = []
for sujeto in subjetos:
    for sesion in sesiones:
        ruta = f"{base_path}/{sujeto}/{sesion}/anat/{sujeto}_{sesion}_T1w_brain_registered.nii.gz"
        image_paths.append(ruta)

random.shuffle(image_paths)

train_ratio = 0.75
train_size = int(len(image_paths) * train_ratio)
train_paths = image_paths[:train_size]
test_paths = image_paths[train_size:]

def load_and_normalize_image(image_path):
    image = nib.load(image_path).get_fdata()
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    image = torch.tensor(image)
    mean = torch.tensor([0.485]).view(1, 1, 1, 1)
    std = torch.tensor([0.229]).view(1, 1, 1, 1)
    image = (image - mean) / std

    return image

train_images = torch.stack([load_and_normalize_image(p) for p in train_paths])
test_images = torch.stack([load_and_normalize_image(p) for p in test_paths])

batch_size = 1

train_dataloader = DataLoader(TensorDataset(train_images), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(TensorDataset(test_images), batch_size=batch_size, shuffle=False)
for batch in train_dataloader:
  print(f"Tamaño del batch de entrenamiento: {batch[0].shape}")
for batch in test_dataloader:
  print(f"Tamaño del batch de prueba: {batch[0].shape}")
train_dataloader, test_dataloader

Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de entrenamiento: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de prueba: torch.Size([1, 1, 91, 109, 91])
Tamaño del batch de prueba: torch.Size([1, 1, 91, 109, 91])


(<torch.utils.data.dataloader.DataLoader at 0x7b5dd7c9d8a0>,
 <torch.utils.data.dataloader.DataLoader at 0x7b5dd7c9f8e0>)

In [15]:
import nibabel as nib
import torch
import numpy as np
import random from torch.utils.data import DataLoader, TensorDataset

def load_and_normalize_image(image_path):
    image = nib.load(image_path).get_fdata()
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)
    image = torch.tensor(image)
    mean = torch.tensor([0.485]).view(1, 1, 1, 1, 1)
    std = torch.tensor([0.229]).view(1, 1, 1, 1, 1)
    image = (image - mean) / std
    return image

subjetos = ["sub-RC4101", "sub-RC4103", "sub-RC4106", "sub-RC4107"]
sesiones = ["ses-1", "ses-2"]
base_path = "drive/MyDrive/Colab Notebooks/output/derivatives/preprocessed"

image_paths = []
for sujeto in subjetos:
    for sesion in sesiones:
        ruta = f"{base_path}/{sujeto}/{sesion}/anat/{sujeto}_{sesion}_T1w_brain_registered.nii.gz"
        image_paths.append(ruta)

random.shuffle(image_paths)

train_ratio = 0.75
train_size = int(len(image_paths) * train_ratio)
train_paths = image_paths[:train_size]
test_paths = image_paths[train_size:]

train_images = torch.stack([load_and_normalize_image(p) for p in train_paths])
test_images = torch.stack([load_and_normalize_image(p) for p in test_paths])

train_labels = torch.tensor([random.randint(0, 1) for _ in train_paths], dtype=torch.long)
test_labels = torch.tensor([random.randint(0, 1) for _ in test_paths], dtype=torch.long)

batch_size = 1

train_dataloader = DataLoader(TensorDataset(train_images, train_labels), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(TensorDataset(test_images, test_labels), batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    print(f"Tamaño del batch de entrenamiento: {batch[0].shape}")
    break

for batch in test_dataloader:
    print(f"Tamaño del batch de prueba: {batch[0].shape}")
    break

train_dataloader, test_dataloader



In [ ]:
import torch.nn as nn
import torch.optim as optim

model_dict = { 'efficientnet_b0': (torchvision.models.efficientnet_b0, torchvision.models.EfficientNet_B0_Weights.DEFAULT, 1280),
              'vgg16': (torchvision.models.vgg16, torchvision.models.VGG16_Weights.DEFAULT, 25088),
               'vgg19': (torchvision.models.vgg19, torchvision.models.VGG19_Weights.DEFAULT, 25088) }
def prepare_model(model_name):
    model_fn, weights, in_features = model_dict[model_name]
    model = model_fn(weights=weights)
    model.features[0][0] = nn.Conv3d(
        in_channels=1,
        out_channels=model.features[0][0].out_channels,
        kernel_size=model.features[0][0].kernel_size,
        stride=model.features[0][0].stride,
        padding=model.features[0][0].padding,
        bias=False
    )

    for param in model.features.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(in_features=in_features, out_features=2)  # 2 salidas para las dos clases
    )

    return model



In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    return model

def set_seeds(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seeds()

model_names = ['efficientnet_b0', 'vgg16', 'vgg19']
num_epochs = 5
criterion = nn.CrossEntropyLoss()

for model_name in model_names:
    print(f"Preparando y entrenando el modelo: {model_name}")
    model = prepare_model(model_name)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model = train_model(model, train_dataloader, criterion, optimizer, num_epochs)
    print(f"Entrenamiento del modelo {model_name} completado.\n")
